# Super Wav2Lip

This Colab project is based on [Wav2Lip-GFPGAN](https://github.com/ajay-sainy/Wav2Lip-GFPGAN), but updates the requirements.txt (to function properly) and updates Colab file for ease of use.

In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model

!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'

!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
#!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O {wav2lipPath}'/checkpoints/wav2lip.pth'

!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1 # The process will fail without downgrading librosa
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

from IPython.display import clear_output
clear_output()

print("Installation complete.")

## 2. LipSync on Your Video File

In [ ]:
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp


def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()

#@markdown ### Select an uploading method
upload_method = "Upload" #@param ["Upload", "Custom Path"]
inputVideo = 'vid.mp4' #@param{type:"string"}


# remove previous input video
if os.path.isfile(basePath + '/inputs/'+ inputVideo):
    os.remove(basePath + '/inputs/'+ inputVideo)

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, basePath + '/inputs/'+ inputVideo)
    PATH_TO_YOUR_VIDEO = basePath + '/inputs/'+ inputVideo

elif upload_method == 'Custom Path':
    # Mount Google Drive if it's not already mounted
    #if not os.path.isdir("/content/drive/MyDrive"):
    #    drive.mount('/content/drive', force_remount=True)

    #@markdown ``Add the full path to your video on your Gdrive `` 👇
    PATH_TO_YOUR_VIDEO = '/content/drive/MyDrive/test.mp4' #@param {type:"string"}
    if not os.path.isfile(PATH_TO_YOUR_VIDEO):
        print("ERROR: File not found!")
        raise SystemExit(0)

#@markdown <font color="orange">Notes:

#@markdown <font color="orange">. ``If your uploaded video is 1080p or higher resolution, this cell will resize it to 720p.``

#@markdown <font color="orange">. ``Do not upload videos longer than 60 seconds.``

#@markdown ___

video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 basePath + '/inputs/'+ inputVideo")
    PATH_TO_YOUR_VIDEO = basePath + '/inputs/'+ inputVideo
    print("Video resized to 720p")
else:
    print("No resizing needed")

if upload_method == "Upload":
  clear_output()
  print("Input Video")
  showVideo(PATH_TO_YOUR_VIDEO)
else:
    if os.path.isfile(PATH_TO_YOUR_VIDEO):
        shutil.copyfile(PATH_TO_YOUR_VIDEO, basePath + '/inputs/'+ inputVideo)
        print("Input Video")
        showVideo(PATH_TO_YOUR_VIDEO)


In [ ]:
#@title STEP3: Select Audio (Record, Upload from local drive or Gdrive)
import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Upload' #@param ['Record', 'Upload']
inputAudio = 'audio.mp3' #@param{type:"string"}


# remove previous input audio
if os.path.isfile(basePath + '/inputs/'+ inputAudio):
    os.remove(basePath + '/inputs/'+ inputAudio)


def displayAudio():
  display(Audio(basePath + '/inputs/'+ inputAudio))

if upload_method == 'Record':
  audio, sr = get_audio()
  import scipy
  scipy.io.wavfile.write(basePath + '/inputs/'+ inputAudio, sr, audio)

elif upload_method == 'Upload':
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes.'.format(
        name=fn, length=len(uploaded[fn])))

  # Consider only the first file
  PATH_TO_YOUR_AUDIO = str(list(uploaded.keys())[0])

  # Load audio with specified sampling rate
  import librosa
  audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

  # Save audio with specified sampling rate
  import soundfile as sf
  file_format = 'mp3' #@param{type:"string"}
  #file_format = 'wav' #@param ['wav', 'mp3', 'ogg']
  sf.write(basePath + '/inputs/'+ inputAudio, audio, sr, format=file_format)

  clear_output()
  displayAudio()

else: # Custom Path
  from google.colab import drive
  drive.mount('/content/drive')
  #@markdown ``Add the full path to your audio on your Gdrive`` 👇
  PATH_TO_YOUR_AUDIO = '/content/drive/MyDrive/test.wav' #@param {type:"string"}

  # Load audio with specified sampling rate
  import librosa
  audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

  # Save audio with specified sampling rate
  import soundfile as sf
  sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')

  clear_output()
  displayAudio()


## 4. Synchronize Video and Speech

In [ ]:
#@markdown Adjust the parameters below then run the code block to synthesize the speech onto your video file:

import os
outputPath = basePath + '/outputs' 
inputAudio = 'audio.mp3' #@param{type:"string"}
inputAudioPath = basePath + '/inputs/' + inputAudio  
inputVideo = 'video.mov' #@param{type:"string"}
inputVideoPath = basePath + '/inputs/'+ inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'  
model = "wav2lip" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}


if not os.path.exists(outputPath):
  os.makedirs(outputPath)

from IPython.display import clear_output
clear_output()

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}



#print("Video synthesis complete.")

## 3. Boost the Resolution of the Synthesized Video



In [ ]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("The video is "+str(fps)+" FPS.")

batch = 0
batchSize = 1300
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

from IPython.display import clear_output
clear_output()

print("Video upscaling complete.")

In [ ]:
#@title STEP4: Download 
from google.colab import files
#@markdown ``Path to file `` 👇
inputFile= '/outputs/' #@param {type:"string"}
PATH_TO_YOUR_File = basePath + inputFile

    if not os.path.isfile(PATH_TO_YOUR_File):
        print("ERROR: File not found!")
        raise SystemExit(0)

    else:
        files.download(PATH_TO_YOUR_File)


## 5. Clear Cached Files

Run this block once you've downloaded your final video file. This will empty /inputs and /outputs, so you can start again, fresh.


In [ ]:
%cd /content/wav2lip-HD/

#@markdown Choose whether to remove both inputs and outputs, or just one of the two. You may want to preserve inputs if you are only changing one of the two inputs. 

removeInputs = True #@param {type:"boolean"}
removeOutputs = True #@param {type:"boolean"}

if removeInputs == True:
  %rm inputs/*
if removeOutputs == True:
  %rm outputs/frames/*
  %rm outputs/restored_imgs/*
  %rm outputs/*


from IPython.display import clear_output
clear_output()

print("Cleared cached files.")
